## Shake detection

### Building a classifier for detecting Shake gestures using data from phone sensors

#### Problem and dataset description

The goal is to create a classifier that can be implemented easily on streaming sensor data to detect Shake gesture.

There are 3 data sets:

1. a.lbl.csv, a.sensor.csv
2. m.lbl.csv, m.sensor.csv
3. p.lbl.csv, p.sensor.csv

Each set has two files: **sensor.csv** and **lbl.csv**.

**sensor.csv** has data from various phone sensors with the below information as columns:

* **timestamp(ms)**: Unix Timestamp in milliseconds.
* **acceleration_x(g)**: Acceleration along phone's x axis (measured in g)
* **acceleration_y(g)**: Acceleration along phone's y axis (measured in g)
* **acceleration_z(g)**: Acceleration along phone's z axis (measured in g)
* **roll(rad)**: Roll angle representing phone's attitude (orientation)
* **pitch(rad)**: Pitch angle representing phone's attitude (orientation)
* **yaw(rad)**: Yaw angle representing phone's attitude (orientation)
* **angular_velocity_x(rad/sec)**: Angular velocity around x axis (passing thru phones center along smaller edge)
* **angular_velocity_y(rad/sec)**: Angular velocity around y axis (passing thru phones center along longer edge)
* **angular_velocity_z(rad/sec)**: Angular velocity around z axis (passing thru phones center emerging out of phone )

The data is recorded every 100 ms (10Hz frequency).

**lbl.csv** contains 2 columes:

* **timestamp(ms)**: Unix Timestamp in milliseconds.
* **label**: 2 possible types of labels as given below.

Possible types of labels in **lbl.csv**:

1. Label **0** : Represents start of Shake gesture
2. Label **1** : Indicates end of Shake gesture

The Goal is to create a classifier that can be implemented easily on streaming sensor data to detect Shake gesture.

### Setting configuration and loading libraries

In [1]:
# Configuration
trainSetRatio <- 1
modelNameStr <- "shakeDetectionClassifier"
classLabel <- "label"

# Load required libraries
library(caret)
#library(rattle)

print("Done")

Loading required package: lattice
Loading required package: ggplot2


[1] "Done"


### Reading input datasets

In [2]:
# Reading in the sensor and label dataset
print("Reading input datasets ...")

a.sensor <- read.csv("data/a.sensor.csv", colClasses="numeric")
m.sensor <- read.csv("data/m.sensor.csv", colClasses="numeric")
p.sensor <- read.csv("data/p.sensor.csv", colClasses="numeric")

a.lbl <- read.csv("data/a.lbl.csv", colClasses="numeric")
m.lbl <- read.csv("data/m.lbl.csv", colClasses="numeric")
p.lbl <- read.csv("data/p.lbl.csv", colClasses="numeric")

print("Summary of sensor data:")
summary(a.sensor)
summary(m.sensor)
summary(p.sensor)

print("Summary of label data:")
summary(a.lbl)
summary(m.lbl)
summary(p.lbl)

print("Done.")

[1] "Reading input datasets ..."
[1] "Summary of sensor data:"


 timestamp.ms.       acceleration_x.g.  acceleration_y.g.   acceleration_z.g. 
 Min.   :1.397e+12   Min.   :-0.57910   Min.   :-0.974300   Min.   :-1.68920  
 1st Qu.:1.397e+12   1st Qu.: 0.00150   1st Qu.: 0.003100   1st Qu.:-0.00860  
 Median :1.397e+12   Median : 0.00320   Median : 0.005100   Median :-0.00600  
 Mean   :1.397e+12   Mean   : 0.00452   Mean   : 0.005245   Mean   :-0.00663  
 3rd Qu.:1.397e+12   3rd Qu.: 0.00500   3rd Qu.: 0.007000   3rd Qu.:-0.00220  
 Max.   :1.397e+12   Max.   : 1.50560   Max.   : 1.282100   Max.   : 0.97860  
   roll.rad.         pitch.rad.         yaw.rad.      
 Min.   :-0.3519   Min.   :-1.4522   Min.   :-3.1409  
 1st Qu.: 0.3133   1st Qu.:-0.6624   1st Qu.: 0.8498  
 Median : 0.3193   Median :-0.6362   Median : 1.2403  
 Mean   : 0.6234   Mean   :-0.6615   Mean   : 1.2405  
 3rd Qu.: 1.3728   3rd Qu.:-0.6143   3rd Qu.: 1.9628  
 Max.   : 2.4940   Max.   :-0.3796   Max.   : 3.1401  
 angular_velocity_x.rad.sec. angular_velocity_y.rad.sec.
 Min.

 timestamp.ms.       acceleration_x.g.  acceleration_y.g.  acceleration_z.g.
 Min.   :1.397e+12   Min.   :-7.34380   Min.   :-5.75090   Min.   :-7.3489  
 1st Qu.:1.397e+12   1st Qu.:-0.03190   1st Qu.:-0.02270   1st Qu.:-0.0942  
 Median :1.397e+12   Median : 0.00180   Median : 0.00290   Median :-0.0025  
 Mean   :1.397e+12   Mean   : 0.02902   Mean   : 0.07933   Mean   :-0.0314  
 3rd Qu.:1.397e+12   3rd Qu.: 0.06980   3rd Qu.: 0.07100   3rd Qu.: 0.0564  
 Max.   :1.397e+12   Max.   : 7.66600   Max.   : 8.60060   Max.   : 4.2670  
   roll.rad.         pitch.rad.         yaw.rad.     
 Min.   :-3.1208   Min.   :-1.5570   Min.   :-3.141  
 1st Qu.:-0.4437   1st Qu.: 0.0083   1st Qu.:-1.915  
 Median :-0.0766   Median : 0.2682   Median :-1.514  
 Mean   :-0.1789   Mean   : 0.2789   Mean   :-0.296  
 3rd Qu.:-0.0177   3rd Qu.: 0.4857   3rd Qu.: 2.206  
 Max.   : 3.1363   Max.   : 1.5416   Max.   : 3.141  
 angular_velocity_x.rad.sec. angular_velocity_y.rad.sec.
 Min.   :-28.17080        

 timestamp.ms.       acceleration_x.g.  acceleration_y.g.  acceleration_z.g. 
 Min.   :1.397e+12   Min.   :-3.01870   Min.   :-2.91270   Min.   :-2.69040  
 1st Qu.:1.397e+12   1st Qu.:-0.00330   1st Qu.:-0.00270   1st Qu.: 0.00560  
 Median :1.397e+12   Median :-0.00020   Median :-0.00020   Median : 0.00860  
 Mean   :1.397e+12   Mean   :-0.05438   Mean   : 0.02223   Mean   : 0.02437  
 3rd Qu.:1.397e+12   3rd Qu.: 0.00180   3rd Qu.: 0.00240   3rd Qu.: 0.01320  
 Max.   :1.397e+12   Max.   : 2.34530   Max.   : 2.76790   Max.   : 2.58660  
   roll.rad.          pitch.rad.          yaw.rad.      
 Min.   :-3.14120   Min.   :-1.55500   Min.   :-3.1411  
 1st Qu.:-0.16330   1st Qu.:-0.11130   1st Qu.:-1.0503  
 Median : 0.00510   Median :-0.10080   Median : 1.5927  
 Mean   :-0.04836   Mean   :-0.09124   Mean   : 0.7227  
 3rd Qu.: 0.00570   3rd Qu.: 0.02470   3rd Qu.: 2.0301  
 Max.   : 3.13920   Max.   : 1.51790   Max.   : 3.1414  
 angular_velocity_x.rad.sec. angular_velocity_y.rad.sec

[1] "Summary of label data:"


 timestamp.ms.       label..0.start.1.end.2.cancel.
 Min.   :1.397e+12   Min.   :0.0                   
 1st Qu.:1.397e+12   1st Qu.:0.0                   
 Median :1.397e+12   Median :0.5                   
 Mean   :1.397e+12   Mean   :0.5                   
 3rd Qu.:1.397e+12   3rd Qu.:1.0                   
 Max.   :1.397e+12   Max.   :1.0                   

 timestamp.ms.       label..0.start.1.end.2.cancel.
 Min.   :1.397e+12   Min.   :0.0                   
 1st Qu.:1.397e+12   1st Qu.:0.0                   
 Median :1.397e+12   Median :0.5                   
 Mean   :1.397e+12   Mean   :0.5                   
 3rd Qu.:1.397e+12   3rd Qu.:1.0                   
 Max.   :1.397e+12   Max.   :1.0                   

 timestamp.ms.       label..0.start.1.end.2.cancel.
 Min.   :1.397e+12   Min.   :0.0                   
 1st Qu.:1.397e+12   1st Qu.:0.0                   
 Median :1.397e+12   Median :0.5                   
 Mean   :1.397e+12   Mean   :0.5                   
 3rd Qu.:1.397e+12   3rd Qu.:1.0                   
 Max.   :1.397e+12   Max.   :1.0                   

[1] "Done."


### Preparing data for analysis

#### Data transformations

Convert the 'lbl' datasets into 'start_time,end_time' format. This is for easier implementation of the logic for creating a labelled training dataset.

In [3]:
## Transform label dataset into "<startime>,<endtime>" format
transformLabelData <- function(lbl){
    lbl.modified <- cbind(lbl[lbl[, 2] == 0, 1], lbl[lbl[, 2] == 1, 1])
    return(lbl.modified)
}

## Transforming the label dataset into 'start_time,end_time' format
print("Transforming label dataset into 'start_time,end_time' format ...")
a.lbl.modified <- transformLabelData(a.lbl)
m.lbl.modified <- transformLabelData(m.lbl)                                              
p.lbl.modified <- transformLabelData(p.lbl)

print("Summary of label data after transformation:")
summary(a.lbl.modified)

print("Done")

[1] "Transforming label dataset into 'start_time,end_time' format ..."
[1] "Summary of label data after transformation:"


       V1                  V2           
 Min.   :1.397e+12   Min.   :1.397e+12  
 1st Qu.:1.397e+12   1st Qu.:1.397e+12  
 Median :1.397e+12   Median :1.397e+12  
 Mean   :1.397e+12   Mean   :1.397e+12  
 3rd Qu.:1.397e+12   3rd Qu.:1.397e+12  
 Max.   :1.397e+12   Max.   :1.397e+12  

[1] "Done"


#### Creating labelled dataset

Create labelled training dataset for each of the sensor datasets separately and then combine them. '**apply**' and '**sapply**' functions are used for this and no 'for' loops since 'for' loops are really inefficient in R. The 'apply' functions could also be parallelized using their multicore versions from 'parallel' package in R and even further optimized by multiprocessing or even using frameworks like Spark for large datasets.

In [4]:
## Create a labelled training set from sensor and label datasets
createTrainingDataset <- function(sensor, lbl.modified){
    label <- sapply(sensor[, 1], 
                    function(ts)
                        any(apply(lbl.modified, 1, 
                                  function(shake_ts, test_ts)
                                      test_ts >= shake_ts[1] && 
                                      test_ts <= shake_ts[2], test_ts=ts)))
    label[label == TRUE] <- "SHAKE"
    label[label == FALSE] <- "NO_SHAKE"
    sensor.labelled <- cbind(sensor[, -1], label)
    return(sensor.labelled)
}
                                  
# Create a labelled training datasets
print("Creating labelled training dataset ...")
a.sensor.labelled <- createTrainingDataset(a.sensor, a.lbl.modified)
m.sensor.labelled <- createTrainingDataset(m.sensor, m.lbl.modified)
p.sensor.labelled <- createTrainingDataset(p.sensor, p.lbl.modified)

# Combining the labelled training datasets
sensor.labelled <- rbind(a.sensor.labelled, m.sensor.labelled, p.sensor.labelled)

print("Summary of labelled training dataset:")
summary(sensor.labelled)

[1] "Creating labelled training dataset ..."
[1] "Summary of labelled training dataset:"


 acceleration_x.g.  acceleration_y.g.  acceleration_z.g.     roll.rad.      
 Min.   :-7.34380   Min.   :-5.75090   Min.   :-7.348900   Min.   :-3.1412  
 1st Qu.:-0.00110   1st Qu.:-0.00090   1st Qu.:-0.008300   1st Qu.:-0.0073  
 Median : 0.00170   Median : 0.00290   Median :-0.001100   Median : 0.0173  
 Mean   :-0.01778   Mean   : 0.02224   Mean   : 0.003558   Mean   : 0.2283  
 3rd Qu.: 0.00480   3rd Qu.: 0.00710   3rd Qu.: 0.009400   3rd Qu.: 0.3194  
 Max.   : 7.66600   Max.   : 8.60060   Max.   : 4.267000   Max.   : 3.1392  
   pitch.rad.         yaw.rad.       angular_velocity_x.rad.sec.
 Min.   :-1.5570   Min.   :-3.1411   Min.   :-28.170800         
 1st Qu.:-0.6355   1st Qu.: 0.3465   1st Qu.: -0.004800         
 Median :-0.1668   Median : 1.3916   Median :  0.000200         
 Mean   :-0.2924   Mean   : 0.8165   Mean   :  0.002184         
 3rd Qu.:-0.0644   3rd Qu.: 2.0131   3rd Qu.:  0.004800         
 Max.   : 1.5416   Max.   : 3.1414   Max.   : 22.085100         
 angul

### Creating train and test datasets 

Split the labelled dataset into train and test set based on a pre-defined ratio. For 80:20 split, 80% of the data from each 'class label' is selected randomly for training and the rest of testing.

In [5]:
# Split data into training and test set
trainSet = NULL
testSet = NULL

print("Spliting data into train set and test set ...")
if(trainSetRatio == 1){
    trainSet <- sensor.labelled
    testSet <- sensor.labelled

} else if(trainSetRatio < 1){
    inTrain <- createDataPartition(sensor.labelled[, classLabel], p = trainSetRatio, list=FALSE)

    trainSet <- sensor.labelled[inTrain,]
    testSet <- sensor.labelled[-inTrain,]
}
    
print("Done")

[1] "Spliting data into train set and test set ..."
[1] "Done"


### Training the classifier

A '**decision-tree**' based classifier is trained using '**recursive partitioning**' ('rpart' method in R). The choice of using a decision tree based algorithm is because decision trees would be much more efficient for shake detection on incoming streaming data. Decsion trees will be simple to interpret and can be converted into rules for  matching on input data if needed. Choice of 'recursive partitioning'/ 'rpart' from among the available tree based algorithms is arbitrary.

#### Hyper-parameter tuning using Grid Search

'rpart' has just one hyper-parameter which is the '**complexity parameter**' - 'cp'. The best 'cp' is selected by doing  Grid Search using '**repeated cross-validation**' (10 fold cross validation repeated 10 times) on a range of 'cp' values. The 'cp' value which gives the best 'F1-Score' from the 'repeated cross-validation' is selected. The reasoning behind using '**F1-score**' here is explained below in 'Design Considerations'.

#### Design Considerations:  Data is heavily skewed!

After creating a labelled dataset for training the classifier, it is observed that **only 1.5% of the training data consists of measurements while the phone was 'Shaking' (Positive class for the Classifier)**. This means that the data is heavily skewed. This makes the classification problem more like a '**needle in a haystack**' problem. In this case, the probability of even a random classifier giving a lot of 'True Negatives' is very high. This will affect our evaluation of the classifier if we are not careful enough. For example, **'Accuracy' of the classifier of even a random classifier will be very high because 'True Negatives' will be high**. Hence we have to use evaluation metrics like Precision (Positive Predictive Value) and Recall (Sensitivity/  True Positive Rate) to evaluate the classifier or even selecting parameters in the training phase. Precision, Recall, F1-Score or some other similar 'True Negative' independent metrics should be used to select hyper-parameters values for a classification algorithm for solving such a problem. Here, I've used 'recursive partitioning' algorithm to build a classifier and used 'repeated cross-validation' to select the 'complexity parameter' that maximizes 'F1-Score'. **A custom function is written for this which is used by 'trainControl' in 'caret' package in R for training the classifier**.

**Note:** Another approach to handle skewed datasets is to do up/ down sampling i.e., either increase the number of positive samples by replication (as is or with slight variations) or decrease th number of negative samples by taking only a random subset of the negative class set with size roughly equal to the size of the positive class set. Code for this is commented out as it is not used in this evaluation.

In [6]:
# Create classLabel ~ predictor1 + predictor2 .. string
predictors <- colnames(sensor.labelled)[-ncol(sensor.labelled)]

# Print predictors
print("Predictors:")
print(predictors)

# Uncomment relevant parts of below code for up/ down sampling training data

# print(paste("Before up/ down sampling: nrow(trainSet):", 
#       nrow(trainSet), "nrow(testSet):", nrow(testSet)))

# trainSet <- upSample(trainSet[, predictors], 
#                     as.factor(trainSet[, classLabel]), 
#                     list=FALSE, yname=classLabel)
# trainSet <- downSample(trainSet[, predictors], 
#                        as.factor(trainSet[, classLabel]), 
#                        list=FALSE, yname=classLabel)

# print(paste ("After up/ down sampling: nrow(trainSet):", nrow(trainSet), "nrow(testSet):", nrow(testSet)))

# Training the classifier
print("Training the classifier ...")

# Custom summary function for trainControl
trainControlSumFuncCustom <- function(data, lev=NULL, model=NULL){
    if(!all(levels(data[, "pred"]) == levels(data[, "obs"]))){
        print("ERROR: Levels of observed and predicted data do not match")
        q()
    }
    
    precision <- posPredValue(data[, "pred"], data[, "obs"], 
                              positive="SHAKE")
    
    recall <- sensitivity(data[, "pred"], data[, "obs"], 
                          positive="SHAKE")
    
    f1score <- 2 * ((precision * recall) / (precision + recall))

    out <- c(precision, recall, f1score)
    names(out) <- c("Precision", "Recall", "F1-Score")

    return(out)
}

fitControl <- trainControl(method="repeatedcv", 
                           number=10, repeats=10, 
                           classProbs=TRUE, 
                           summaryFunction=trainControlSumFuncCustom, 
                           selectionFunction="best")

rpart.grid <- expand.grid(cp=c(0.002, 0.0025, 0.003, 0.0035, 0.004, 0.0045, 0.005, 0.0055, 0.006, 0.0065))

rpartFit <- train(trainSet[, predictors], 
                  as.factor(as.character(trainSet[, classLabel])), 
                  method="rpart", 
                  trControl=fitControl, 
                  metric="F1-Score", 
                  tuneGrid = rpart.grid)

# Print the classifier details
print(rpartFit)
print("Done")

[1] "Predictors:"
[1] "acceleration_x.g."           "acceleration_y.g."          
[3] "acceleration_z.g."           "roll.rad."                  
[5] "pitch.rad."                  "yaw.rad."                   
[7] "angular_velocity_x.rad.sec." "angular_velocity_y.rad.sec."
[9] "angular_velocity_z.rad.sec."
[1] "Training the classifier ..."


Loading required package: rpart


CART 

163869 samples
     9 predictor
     2 classes: 'NO_SHAKE', 'SHAKE' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 147482, 147482, 147482, 147482, 147482, 147482, ... 
Resampling results across tuning parameters:

  cp      Precision  Recall     F1-Score 
  0.0020  0.6831579  0.2500392  0.3648623
  0.0025  0.6964482  0.2358824  0.3513641
  0.0030  0.7063529  0.2282745  0.3436809
  0.0035  0.7078026  0.2204706  0.3350840
  0.0040  0.7108699  0.2136863  0.3273557
  0.0045  0.7106366  0.2083137  0.3208771
  0.0050  0.7061223  0.2041176  0.3153718
  0.0055  0.7026750  0.1976078  0.3070021
  0.0060  0.7053750  0.1949412  0.3039943
  0.0065  0.7053563  0.1916471  0.2998883

F1-Score was used to select the optimal model using  the largest value.
The final value used for the model was cp = 0.002.
[1] "Done"


### Evaluation

The classifier model is evaluated using the test dataset. Output includes predicted probabilies for each class for each sample data, confusion matrix, and other standard evaluation metrics.

In [7]:
# Predict classes without probabilities on test data
predClasses <- predict(rpartFit, newdata=testSet[, predictors])

# Print prediction details on test data
# print("Predictions:")
# print(predClasses)

# Predict classes with probabilities on test data
# predClassesProbs <- predict(rpartFit, newdata=testSet[, predictors], type="prob")

# Print prediction details on test data with probabilities
# print("Class Probabilities:")
# predClassesProbs

# Print confusion matrix
print("Confusion matrix")
confusionMatrix(data=predClasses, positive="SHAKE", as.factor(testSet[, classLabel]))

# Plot classification tree
# fancyRpartPlot(rpartFit$finalModel)

print("Done.")

[1] "Confusion matrix"


Confusion Matrix and Statistics

          Reference
Prediction NO_SHAKE  SHAKE
  NO_SHAKE   161071   1797
  SHAKE         248    753
                                         
               Accuracy : 0.9875         
                 95% CI : (0.987, 0.9881)
    No Information Rate : 0.9844         
    P-Value [Acc > NIR] : < 2.2e-16      
                                         
                  Kappa : 0.419          
 Mcnemar's Test P-Value : < 2.2e-16      
                                         
            Sensitivity : 0.295294       
            Specificity : 0.998463       
         Pos Pred Value : 0.752248       
         Neg Pred Value : 0.988967       
             Prevalence : 0.015561       
         Detection Rate : 0.004595       
   Detection Prevalence : 0.006109       
      Balanced Accuracy : 0.646878       
                                         
       'Positive' Class : SHAKE          
                                         

[1] "Done."
